In [4]:
import os
import dotenv
# import required libraries
from vidgear.gears import CamGear

In [1]:



from collections import deque
import os
import dotenv
# import required libraries
from vidgear.gears import CamGear
from vidgear.gears import WriteGear
import cv2
import logging
from matplotlib import pyplot as plt
import time
logging.basicConfig(level=logging.INFO, format="[%(asctime)s] [%(levelname)s] %(message)s")


project_root_dir = 'E:\\p\\ai-cam'
os.chdir(project_root_dir)
assert os.path.exists(".env")
# Load secret keys from .env file
dotenv.load_dotenv()
YOUTUBE_STREAM_KEY = os.getenv("YOUTUBE_STREAM_KEY")

VIDEO_SOURCE="./dev/notebooks/WNMJ4391.MP4"
stream = CamGear(source=VIDEO_SOURCE, logging=True).start()
assert os.path.exists(VIDEO_SOURCE)
# define video source

# Open stream

# define required FFmpeg optimizing parameters for your writer
# [NOTE]: Added VIDEO_SOURCE as audio-source
# [WARNING]: VIDEO_SOURCE must contain audio
# define required FFmpeg parameters for your writer
output_params = {
    "-clones": ["-f", "lavfi", "-i", "anullsrc"],
    "-vcodec": "libx264",
    "-preset": "medium",
    "-b:v": "4500k",
    "-bufsize": "512k",
    "-pix_fmt": "yuv420p",
    "-f": "flv",
    "-g": 60,
    "-input_framerate": 30
}


# Define writer with defined parameters and
writer = WriteGear(
    output="rtmp://a.rtmp.youtube.com/live2/{}".format(YOUTUBE_STREAM_KEY),
    logging=True,
    **output_params
)



frames = []

for i in range(1000):
    # read frames from stream
    frame = stream.read()
    if False and i < 10:
        plt.close("all")
        plt.imshow(frame)
        plt.show()
        print(type(frame))
    if frame is None:
        break
    frames.append(frame)

stream.stop()

# loop over
start_time = time.time()
frames_sent = 0
frames_watched = 0
frames_ahead = 0

frames_rest = 10*30 # 30 frames per second 10 seconds of buffering

print("%s frames in memory" %len(frames))
for i in range(100000):
    frame = frames[i % len(frames)]
    writer.write(frame)
    cur = time.time()
    frames_sent += 1
    frames_watched = (cur-start_time)*30
    frames_ahead = frames_sent - frames_watched
    if frames_ahead > frames_rest:
        time.sleep(1/29)
    if i % frames_rest == 0:
        print("frames_sent: %s frames_watched: %s frames_ahead: %s" %(frames_sent, frames_watched, frames_ahead))




# safely close writer
writer.close()

21:34:29 ::    CamGear    ::  DEBUG   :: Enabling Threaded Queue Mode for the current video source!
21:34:29 ::    Helper     :: WARNING  :: Specified directory `E:\p\ai-cam\rtmp:\a.rtmp.youtube.com\live2` doesn't exists or valid.
21:34:29 ::   WriteGear   ::   INFO   :: `rtmp://a.rtmp.youtube.com/live2/bp8e-sdag-dmy1-0wvv-7ubr` isn't a valid system path or directory. Skipped!
21:34:29 ::   WriteGear   ::  DEBUG   :: Output Parameters: `{'-clones': ['-f', 'lavfi', '-i', 'anullsrc'], '-vcodec': 'libx264', '-preset': 'medium', '-b:v': '4500k', '-bufsize': '512k', '-pix_fmt': 'yuv420p', '-f': 'flv', '-input_framerate': 30}`
21:34:29 ::   WriteGear   ::  DEBUG   :: Compression Mode is enabled therefore checking for valid FFmpeg executable.
21:34:29 ::   WriteGear   :: WARNING  :: Optional `-disable_ffmpeg_window` flag is only available on Windows OS with `logging=False`. Discarding!
21:34:29 ::    Helper     ::  DEBUG   :: FFmpeg Windows Download Path: C:\Users\round\AppData\Local\Temp
21:

256 frames in memory
frames_sent: 1 frames_watched: 1.3463973999023438 frames_ahead: -0.34639739990234375
frames_sent: 31 frames_watched: 7.217545509338379 frames_ahead: 23.78245449066162
frames_sent: 61 frames_watched: 11.077237129211426 frames_ahead: 49.922762870788574
frames_sent: 91 frames_watched: 18.94620180130005 frames_ahead: 72.05379819869995
frames_sent: 121 frames_watched: 26.934843063354492 frames_ahead: 94.06515693664551
frames_sent: 151 frames_watched: 35.21859884262085 frames_ahead: 115.78140115737915
frames_sent: 181 frames_watched: 42.83665895462036 frames_ahead: 138.16334104537964
frames_sent: 211 frames_watched: 49.947288036346436 frames_ahead: 161.05271196365356
frames_sent: 241 frames_watched: 57.26613521575928 frames_ahead: 183.73386478424072
frames_sent: 271 frames_watched: 64.40765619277954 frames_ahead: 206.59234380722046
frames_sent: 301 frames_watched: 71.42550230026245 frames_ahead: 229.57449769973755
frames_sent: 331 frames_watched: 79.04202461242676 frames

In [ ]:
import subprocess
import cv2

rtmp_url= "rtmp://a.rtmp.youtube.com/live2/{}".format(YOUTUBE_STREAM_KEY)

# In my mac webcamera is 0, also you can set a video file name instead, for example "/home/user/demo.mp4"
cap = cv2.VideoCapture(VIDEO_SOURCE)

# gather video info to ffmpeg
fps = int(cap.get(cv2.CAP_PROP_FPS))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

print(fps, width, height)
command = 'ffmpeg -loglevel debug -threads:v 2 -threads:a 8 -filter_threads 2 
       -thread_queue_size 512  -f dshow -i video="HP Wide Vision HD" 
       -f dshow -i audio="Microphone Array (Realtek Audio)" -pix_fmt yuv420p 
       -c:v libx264 -qp:v 19 -profile:v high -rc:v cbr_ld_hq -level:v 4.2 
       -r:v 60 -g:v 120 -bf:v 3 -refs:v 16 
       -f flv %s' % rtmp_url

# command and params for ffmpeg
command = ['ffmpeg',
           '-y',
           '-f', 'rawvideo',
           '-vcodec', 'rawvideo',
           '-pix_fmt', 'bgr24',
           '-s', "{}x{}".format(width, height),
           '-r', str(fps),
           '-i', '-',
           '-c:v', 'libx264',
           '-pix_fmt', 'yuv420p',
           '-preset', 'ultrafast',
           '-f', 'flv',
           rtmp_url]

print(" ".join(command))

# using subprocess and pipe to fetch frame data
p = subprocess.Popen(command, stdin=subprocess.PIPE)


while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        print("frame read failed")
        break
    plt.close("all")
    plt.imshow(frame)
    plt.show()
    # YOUR CODE FOR PROCESSING FRAME HERE
    # write to pipe
    p.stdin.write(frame.tobytes())

p.stdin.flush()


